# **10.** RESTFUL API

### **10.1.** Overview

* REpresentational State Transfer
* APIs are endpoints from / to the server (back end)
* These end points can be used to get data from or send data to them

* Restful APIs revolve around 3 concepts:
    1. **Resources:** Objects available on the web that can be accessed using URLs
    2. **Resource Representations:** How the data returned from the server will be represented
        * HTML, XML or JSON
    3. **HTTP Methods:** Methods that determine the properties and functionalities of each endpoint
        * GET, POST, PUT, PATCH, DELETE

### **10.2.** Organizing Rest API

In [1]:
# app_name
# |---api/
# |   |---__init__.py
# |   |---authentication.py
# |   |---parsers.py
# |   |---permissions.py
# |   |---renderers.py
# |   |---serializers.py
# |   |---validators.py
# |   |---views.py
# |   |---viewsets.py

* Routers and url paths are placed in the app's urls.py module

### **10.2.** Installing Django Rest Framework

* Installing django rest framework

In [2]:
# pipenv install djangorestframework

* Adding django rest framework to INSTALLED_APPS

In [3]:
THIRD_PARTY_APPS = ["rest_framework"]

### **10.3.** Django Rest Serializers

* Serializers are used to convert between model instance to a python dictionary
* Models are the internal representation of an object, Serializers are the external representation of an object
    * API model does not need to be similar to the Data model

* Serializers convert model instance to python dictionary and then JSON object implicitly

* Write serializers in a serializers.py file in each app

In [4]:
# from rest_framework import serializers


# class ModelNameSerializer(serializers.Serializer):
    # serializer_field = serializer.FieldType(required_attributes=)

* Using a custom field name in the serializer instead of the original model field name

In [5]:
# from rest_framework import serializers


# class ModelNameSerializers(serializers.Serializer):
    # custom_field_name = serializers.FieldType(required_attributes=, source="original_field_name")

* Defining a custom serializer field

In [6]:
# from rest_framework import serializers


# class ModelNameSerializer(serializers.Serializer):
    # custom_serializer_field = serializers.SerializerMethodField(method_name="custom_method_name")


    # def custom_method_name(self, model_name):
        # return some_calculations

* Serializing a relationship
* There are four methods for serializing relationships:
    1. Displaying the primary key of the related model
    2. Displaying the string representation of the related model
    3. Displaying a serializer of the related model
    4. Displaying a hyperlink of the related model

In [7]:
# from rest_framework import serializers
# from .models import RelatedModelName


# class RelatedModelNameSerializer(serializers.Serializer):
    # field_name_one = serializers.FieldType(required_attributes=)


# class ModelNameSerializer(serializers.Serializer):
    # related_field_name_one = serializers.PrimaryKeyRelatedField(queryset=RelatedModelName.objects.all())
    # related_field_name_two = serializers.StringRelatedField()
    # related_field_name_three = RelatedModelNameSerializer()
    # related_field_name_four = serializers.HyperlinkRelatedField(
        # queryset=RelatedModelName.objects.all(),
        # view_name="related-model-view-name",
    # )

**NOTE:** When using hyperlinks to display a related model:
1. context={"request": request} must be included in the view
2. pk must be used as a parameter in the urls and views

* Defining model serializer

In [8]:
# from rest_framework import serializers
# from .models import ModelName


# class ModelNameSerializer(serializers.ModelSerializer):
    # class Meta:
        # model = ModelName
        # fields = ["model_field_one", "model_field_two", ...]

In [9]:
# from rest_framework import serializers
# from .models import ModelName, RelatedModelName


# class RelatedModelNameSerializer(serializers.ModelSerializer):
    # class Meta:
        # model = RelatedModelName
        # fields = ["field_one", "field_two", "custom_name_field"]
    
    # custom_name_field = serializers.FieldType(required_attributes=, source="original_field_name")


# class ModelNameSerializer(serializers.ModelSerializer):
    # class Meta:
        # model = ModelName
        # fields = ["field_one", "related_field"]
    
    # related_field = RelatedModelNameSerializer()

**Bad Practice:** Do not user fields = "\_\_all\_\_" since it might expose sensitive info in the serializer

* Making a field read only

In [10]:
# from rest_framework import serializers
# from .models import ModelName


# class ModelNameSerializer(serializers.ModelSerializer):
    # class Meta:
        # model = ModelName
        # fields = ["field_one", "field_two", ...]
    
    # field_one = serializers.FieldType(read_only=True)

**BEST PRACTICE** Implement validation and CRUD on the serializer level

* Managing the data validation

In [11]:
# from rest_framework import serializers
# from .models import ModelName


# class ModelNameSerializer(serializers.ModelSerializer):
    # class Meta:
        # model = ModelName
        # fields = ["field_one", "field_two", ...]
    
    # def validate_field_one(self, value):
        # if value has some problem:
            # raise serializers.ValidationError("message")
        # return value
    
    # def validate(self, data):
        # if data["field_name"] is not correct:
            # raise serializers.ValidationError("message")
        # return data

* Creating an object
* The create method is invoked upon calling the .save() method in a POST request

In [12]:
# from rest_framework import serializers
# from .models import ModelName


# class ModelNameSerializer(serializers.ModelSerializer):
    # class Meta:
        # model = ModelName
        # fields = ["field_one", "field_two", ...]
    
    # def create(self, validated_data):
        # model_instance = ModelName(**validated_data)
        # modification and adjustments on the model_instance
        # model_instance.save()
        # return model_instance

* Updating an object
* The update method is invoked, upon calling the .save() method in a PUT or PATCH request

In [13]:
# from rest_framework import serializers
# from .models import ModelName


# class ModelNameSerializer(serializers.ModelSerializer):
    # class Meta:
        # model = ModelName
        # fields = ["field_one", "field_two", ...]
    
    # def update(self, instance, validated_data):
        # instance.field_name_one = validated_date.get("field_name_one")
        # instance.save()
        # return instance

### **10.4.** Django Rest Views

* Rest API views can be written as functions or classes

#### **10.4.1.** Function Based Views

* Using rest framework's api_view will display **Browsable API** page with the data returned
* api_view takes a list of HTTP methods
    * ["GET", "POST", "PUT", "PATCH", "DELETE"]
    * If a method other than "GET" is used, "GET" must be explicitly mentioned

In [14]:
# from django.shortcuts import get_object_or_404
# from rest_framework.decorators import api_view
# from rest_framework.response import Response
# from .models import ModelName
# from .serializers import ModelNameSerializer


# @api_view()
# def view_name_list(request):
    # query_set = ModelName.objects.all()
    # serializer = ModelNameSerializer(query_set, many=True, context={"request": request})
    # return Response(serializer.data)


# @api_view()
# def view_name_details(request, parameter)
    # object = get_object_or_404(ModelName, parameter=parameter)
    # serializer = ModelNameSerializer(object)
    # return Response(serializer.data)

* **Create Object:** Deserializing the data in order to create an object

In [15]:
# from rest_framework.decorators import api_view
# from rest_framework.response import Response
# from rest_framework.status import HTTP_400_BAD_REQUEST, HTTP_201_CREATED
# from .models import ModelName
# from .serializers import ModelNameSerializer


# @api_view(["GET", "POST"])
# def view_name_list(request):
    # if request.method == "GET":
        # queryset = ModelName.objects.all()
        # serializer = ModelNameSerializer(queryset, many=True)
        # return Response(serializer.data)
    # if request.method == "POST":
        # serializer = ModelNameSerializer(data=request.data)
        # serializer.is_valid(raise_exception=True):
        # serializer.save()
        # return Response(serializer.data, status=HTTP_201_CREATED)

* **Update object:** Deserializing the data in order to update an object

In [16]:
# from django.shortcuts import get_object_or_404
# from rest_framework.decorators import api_view
# from rest_framework.response import Response
# from rest_framework.status import HTTP_200_OK
# from .models import ModelName
# from .serializers import ModelNameSerializer


# @api_view(["GET", "PUT", "PATCH"])
# def view_name_details(request, pk):
    # model_instance = get_object_or_404(ModelName, pk=pk)

    # if request.method == "GET":
        # serializer = ModelNameSerializer(model_instance)
        # return Response(serializer.data)
    # if request.method == "PUT":
        # serializer = ModelNameSerializer(instance=model_instance, data=request.data)
        # serializer.is_valid(raise_exception=True)
        # serializer.save()
        # return Response(serializer.data, status=HTTP_200_OK)

* **Delete object:** Deserializing data in order to delete and object

In [17]:
# from django.shortcuts import get_object_or_404
# from rest_framework.decorators import api_view
# from rest_framework.response import Response
# from rest_framework.status import HTTP_204_NO_CONTENT, HTTP_405_METHOD_NOT_ALLOWED
# from .models import ModelName
# from .serializers import ModelNameSerializer


# @api_view(["GET", "DELETE"])
# def view_name_details(request, pk):
    # model_instance = get_object_or_404(ModelName, pk=pk)

    # if request.method == "GET":
        # serializer = ModelNameSerializer(model_instance)
        # return Response(serializer.data)
    # if request.method == "DELETE":
        # if model_instance can not be deleted:
            # return Response({"error": "message"}, status=HTTP_405_METHOD_NOT_ALLOWED)
        # model_instance.delete()
        # return Response(status=HTTP_204_NO_CONTENT)

#### **10.4.2.** Class Based Views

* Using APIView

In [18]:
# from django.shortcuts import get_object_or_404
# from rest_framework.view import APIView
# from rest_framework.response import Response
# from .models import ModelName
# from .serializers import ModelNameSerializer


# class ModelNameListView(APIView):
    # def get(self, request):
        # queryset = ModelName.objects.all()
        # serializer = ModelNameSerializer(queryset, name=True)
        # return Response(serializer.data, status=)
    
    # def post(self, request):
        # serializer = ModelNameSerializer(data=request.data)
        # serializer.is_valid(raise_exception=True)
        # serializer.save()
        # return Response(serializer.data, status=)


# class ModelNameDetailView(APIView):
    # def put(self, request, pk):
        # instance = get_object_or_404(ModelName, pk=pk)
        # serializer = ModelNameSerializer(instance, data=request.data)
        # serializer.is_valid(raise_exception=True)
        # serializer.save()
        # return Response(serializer.data, status=)

    # def patch(self, request, pk):
        # instance = get_object_or_404(ModelName, pk=pk)
        # serializer = ModelNameSerializer(instance, data=request.data)
        # serializer.is_valid(raise_exception=True)
        # serializer.save()
        # return Response(serializer.data, status=)
    
    # def delete(self, request, pk):
        # instance = get_object_or_404(ModelName, pk=pk)
        # if check for some conditions if necessary:
            # return Response({"error": "message"}, status=)
        # instance.delete()
        # return Response(status=)

* Using Model Mixins
    * APIView's implementation for the http methods is similar to the function based views
    * Mixins encapsulate these repeated implementations into a reusable class

* Using Generic Views
    * Generic views combines the Mixins and APIView

In [19]:
# from rest_framework.generics import ListAPIView, CreateAPIView, RetrieveUpdateDestroyAPIView
# from .models import ModelName
# from .serializers import ModelNameSerializer


# class ModelNameListView(ListAPIView, CreateAPIView):
    # queryset = ModelName.object.select_related().filter().annotate()
    # serializer_class = ModelNameSerializer

    # def get_serializer_context(self):
        # return {"request": self.request}


# class ModelNameDetailView(RetrieveUpdateDestroyAPIView):
    # queryset = ModelName.objects.all()
    # serializer_class = ModelNameSerializer

* Overriding the base functions to add custom logic

In [20]:
# from rest_framework.generics import ListAPIView, CreateAPIView, RetrieveAPIView, UpdateAPIView, DestroyAPIView
# from .models import ModelName
# from .serializers import ModelNameSerializer


# class ModelNameListView(ListAPIView, CreateAPIView):
    # def get_queryset(self):
        # return ModelName.objects.all()
    
    # def get_serializer_class(self):
        # return ModelNameSerializer
    
    # def get_serializer_context(self):
        # return {"request": self.request}


# class ModelNameDetailView(RetrieveUpdateDestroyAPIView):
    # queryset = ModelName.objects.all()
    # serializer_class = ModelNameSerializer

    # def get(self, request, pk):
        # pass
    
    # def delete(self, request, pk):
        # pass

* Changing the name of the lookup field

In [21]:
# from rest_framework.generics import ListCreateAPIView
# from .models import ModelName
# from .serializers import ModelNameSerializer


# class ModelNameListView(ListCreateAPIView):
    # queryset = ModelName.objects.all()
    # serializer_class = ModelNameSerializer
    # lookup_field = "custom_name"

* Overriding base queryset and serializer_class
* The get_queryset implements nested urls

In [22]:
# from rest_framework.generics import ListCreateAPIView, RetrieveUpdateDestroyAPIView
# from .models import ModelName
# from .serializers import ModelNameReadSerializer, ModelNameWriteSerializer

# class ModelNameListAPIView(ListCreateAPIView):
    # def get_queryset(self):
        # return ModelName.objects.filter(related_model_name__uuid=self.kwargs.get("uuid"))

    # def get_serializer_class(self):
        # if self.request.method == "GET":
            # return ModelNameReadSerializer
        # return ModelNameWriteSerializer

* Using non-standard lookup fields

In [23]:
# from django.shortcuts import get_object_or_404
# from rest_framework.generics import RetrieveUpdateDestroyAPIView
# from .models import ModelName
# from .serializers import ModelNameSerializer


# class ModelNameDetailAPIView(RetrieveUpdateDestroyAPIView):
    # queryset = ModelName.objects.all().annotate()
    # serializer_class = ModelNameSerializer

    # def get_object(self):
        # return get_object_or_404(ModelName, uuid=self.kwargs.get("uuid"))

* Using ViewSets
    * Model view sets combines all the mixins and the generic view
    * ModelViewSet allows all HTTP methods
    * ReadOnlyModelViewSet prevents using writing HTTP methods

In [24]:
# from rest_framework.viewsets import ModelViewSet
# from .models import ModelName
# from .serializers import ModelNameSerializer


# class ModelNameViewSet(ModelViewSet):
    # queryset = ModelName.objects.all()
    # serializer_class = ModelNameSerializer

    # def get_serializer_context(self):
        # return {"domain_pk": self.kwargs.get("domain_pk")}
    
    # def list(self, request, *args, **kwargs):
        # Custom get implementation

    # def destroy(self, request, *args, **kwargs):
        # Custom delete implementation

* In a nested routers, to retrieve the domain object:
    * Define **get_serializer_context** method to return the domain identifier
    * Define **get_queryset** to filter sub-domain objects that are within the domain object

* Selecting the allowed HTTP methods

In [25]:
# from rest_framework.viewsets import ModelViewSet
# from .models import ModelName
# from .serializers import ModelNameSerializer


# class ModelNameViewSet(ModelViewSet):
    # queryset = ModelName.objects.all()
    # serializer_class = ModelNameSerializer
    # http_method_names = ["get", "post", "patch", "delete"]

#### **10.4.3.** Filtering, Searching and Ordering

* Simple Filtering
    * By using ?field_name= in front of the resource name in the URL

In [26]:
# from rest_framework.viewsets import ModelViewSet
# from .models import ModelName
# from .serializers import ModelNameSerializer


# class ModelNameViewSet(ModelViewSet):
    # serializer_class = ModelNameSerializer
    
    # def get_queryset(self):
        # queryset = ModelName.objects.all()
        # filter_param = self.request.query_param.get("filter_param")
        # if filter_param:
            # queryset = queryset.filter(filter_param=filter_param)
        # return queryset

* Generic Filtering
    * install django-filter
    * add django_filters to INSTALLED_APPS

In [27]:
# from rest_framework.viewset import ModelViewSet
# from django_filters.rest_framework import DjangoFilterBackend
# from .models import ModelName
# from .serializers import ModelNameSerializer


# class ModelNameViewSet(ModelViewSet):
    # queryset = ModelName.objects.all()
    # serializer_class = ModelNameSerializer
    # filter_backends = [DjangoFilterBackend]
    # filterset_fields = ["field_name_one", "field_name_two"]

* To Customize the filtering lookup fields
    * create a filter.py file
    * in the view use filterset_class instead of filterset_fields

In [2]:
# from django_filters.rest_framework import FilterSet
# from django_filters import CharFilter, DateFilter
# from .models import ModelName


# class ModelNameFilter(FilterSet):
    # field_name_one = CharFilter(field_name="field_name_one", lookup_expr="icontains")
    # field_name_two = DateFilter(field_name="field_name_two", lookup_expr="exact")
    
    # class Meta:
        # model = ModelName
        # fields = ["field_name_one", "field_name_two"]

* Searching

In [29]:
# from rest_framework.viewset import ModelViewSet
# from rest_framework.filters import SearchFilter
# from .models import ModelName
# from .serializers import ModelNameSerializer


# class ModelNameViewSet(ModelViewSet):
    # queryset = ModelName.objects.all()
    # serializer_class = ModelNameSerializer
    # filter_backends = [SearchFilter]
    # search_fields = ["field_name_one", "field_name_two"]

* Ordering

In [30]:
# from rest_framework.viewset import ModelViewSet
# from rest_framework.filters import OrderingFilter
# from .models import ModelName
# from .serializers import ModelNameSerializer


# class ModelNameViewSet(ModelViewSet):
    # queryset = ModelName.objects.all()
    # serializer_class = ModelNameSerializer
    # filter_backends = [OrderingFilter]
    # ordering_fields = ["field_name_one", "field_name_two"]

#### **10.4.4.** Pagination

In [31]:
# from rest_framework.viewset import ModelViewSet
# from rest_framework.pagination import PageNumberPagination
# from .models import ModelName
# from .serializers import ModelNameSerializer


# class ModelNameViewSet(ModelViewSet):
    # queryset = ModelName.objects.all()
    # serializer_class = ModelNameSerializer
    # pagination_class = PageNumberPagination

* To customize the pagination:
    * Create pagination.py
    * set pagination_class equal to the CustomPagination class

In [32]:
# from rest_framework.pagination import PageNumberPagination


# class CustomPagination(PageNumberPagination):
    # page_size = 10

#### **10.4.5.** Actions

* actions can be used to define a function within the view
* actions can be called in the url using their function name:
    * /view_name/action_name/

In [33]:
# from rest_framework.viewsets import ModelViewSet
# from rest_framework.decorators import action
# from .models import ModelName
# from api.serializers import ModelNameSerializer


# class ModelNameViewSet(ModelViewSet):
    # queryset = ModelName.objects.all()
    # serializer_class = ModelNameSerializer

    # @action(detail=False, methods=["GET", "POST"])
    # def action_name(self, request):
        # check the method type
        # query the database
        # apply some logic on the data
        # serializer the query result
        # send a response

### **10.5.** Django Rest URLs

* It is recommended to version the api url 

In [34]:
# from django.urls import path, include

# urlpatterns = [
    # path("api/v1/", include("app_name.urls", namespace="app_name")),
# ]

* URL pattern for using function based and class based (APIView and Generic) views, 

In [35]:
# from django.urls import path
# from .views import view_name_list, view_name_details, ModelNameListView, ModelNameDetailView

# app_name = "the_namespace_provided_in_the_urls"
# urlpatterns = [
    # path(route="view_name/", view=view_name_list, name="view_name:list"),
    # path(route="view_name/<slug|str|int:parameter>/", view=view_name_details, name="view_name:details"),
    # path(route="class_view_name/", view=ModelViewName.as_view(), name="class_view_name:list"),
    # path(route="class_view_name/<type:parameter>/", view=ModelNameDetailView.as_view(), name="class_view_name:detail"),
# ]

* URL pattern for using nested URLs with generic API views

In [36]:
# from django.urls import path
# from .views import ModelNameListAPIView, ModelNameDetailAPIView

# app_name = "namespace"
# url_patterns = [
    # path(
        # route="lv_one/<uuid:uuid>/lv_two/",
        # view=ModelNameListAPIView.as_view(),
        # name="view_name_list",
    # ),
    # path(
        # route="lv_one/<uuid:uuid_1>/lv_two/<uuid:uuid_2>/",
        # view=ModelNameDetailAPIView.as_view,
        # name="view_name_detail",
    # )
# ]

* **Routers** are URL patterns for using ViewSets
    * SimpleRouters:
        * Define the list and details end points for each resources registered
    * DefaultRouter:
        * Define the list and details end points for each resources registered
        * Provides end points for each resources registered
        * Provides .json for each end points

In [37]:
# from django.urls import path, include
# from rest_framework.routers import SimpleRouter
# from .views import ModelNameViewSet, AnotherModelNameViewSet


# router = SimpleRouter()
# router.register(prefix="some_name", viewset=ModelNameViewSet)
# router.register(prefix="some_other_name", viewset=AnotherModelNameViewSet)

# urlpatterns = [
    # path("", include(router.urls)),
# ]

* Nested Routers
    * To use install drf-nested-routers

In [38]:
# from django.urls import path, include
# from rest_framework_nested import routers
# from .views import DomainViewSet, SubDomainViewSet


# router = routers.DefaultRouter()
# router.register(prefix="domain_name", viewset=DomainViewSet, basename="domain-name")

# sub_domain_router = routers.NestedDefaultRouter(parent_router=router, parent_prefix="domain_name", lookup="domain_name")
# sub_domain_router.register(prefix="sub_domain_name", viewset=SubDomainViewSet, basename="domain-sub_domain")

# urlpatterns = [
    # path("", include(router.urls)),
    # path("", include(sub_domain_router.urls)),
# ]

### **10.7.** Rest Framework Settings

* Django rest framework settings can be customized in the settings.py

In [39]:
REST_FRAMEWORK = {
    "COERCE_DECIMAL_TO_STRING": False, # Prevent displaying integer fields as string
    "DEFAULT_PAGINATION_CLASS": "rest_framework.pagination.PageNumberPagination",
    "PAGE_SIZE": 10,
}

### **10.8.** Rest Framework Permissions

* Permissions are used to restrict user access to the resources
* Permissions can be set globally for all views or for individual views

#### **10.8.1.** Global Permissions

In [40]:
REST_FRAMEWORK = {
    "DEFAULT_PERMISSION_CLASSES": ("rest_framework.permissions.IsAdminUser",),
}

#### **10.8.2.** Individual Permissions

In [41]:
# from rest_framework.viewsets import ModelViewSet
# from rest_framework.permissions import IsAuthenticated, IsAdmin
# from rest_framework.decorators import action
# from .models import ModelName
# from .api/serializers import ModelNameSerializer


# class ModelNameViewSet(ModelViewSet):
    # queryset = ModelName.objects.all()
    # serializer_class = ModelNameSerializer
    # permission_classes = [IsAuthenticated,]

    # @action(display=True, methods=["GET", "PUT"], permission_classes=[IsAdmin])
    # def action_name(self, request):
        # action related logic

In [55]:
# from rest_framework.viewsets import ModelViewSet
# from rest_framework.permissions import IsAuthenticated, IsAdmin
# from rest_framework.decorators import action
# from .models import ModelName
# from .api/serializers import ModelNameSerializer


# class ModelNameViewSet(ModelViewSet):
    # serializer_class = ModelNameSerializer

    # def get_queryset(self):
        # if self.request.user.is_staff:
            # return ModelName.objects.all()
        # return ModelName.objects.filter(user_id=self.request.user.id)

In [42]:
# from rest_framework.viewsets import ModelViewSet
# from rest_framework.permissions import IsAuthenticated, IsAdmin
# from .models import ModelName
# from .api/serializers import ModelNameSerializer


# class ModelNameViewSet(ModelViewSet):
    # queryset = ModelName.objects.all()
    # serializer_class = ModelNameSerializer

    # def get_permissions(self):
        # if self.action == "list" or self.request.method == "get":
            # return [IsAdmin()]
        # return [IsAuthenticated()]

#### **10.8.3.** Custom Permissions

* permissions can be created with custom functionalities in permissions.py and be used

In [43]:
# from rest_framework import permissions


# class IsAdminOrReadOnly(permissions.BasePermission):
    # def has_permission(self, request, view):
        # if request.method in permissions.SAFE_METHODS:
            # return True
        # return bool(self.user and self.user.is_staff):

#### **10.8.4.** Custom Model Permissions

* To access the permissions defined in the Meta class of a model

In [44]:
# from rest_framework import permissions


# class PermissionName(permissions.BasePermission):
    # def has_permission(self, request, view):
        # return request.user.has_perm("app_name.model_permissions_code_name")

### **10.9.** Token Based Authentication

* Standard method for authenticating users with RESTful APIs

* The process of this authentication is as follows:
    1. The client login by providing authentication credentials
    2. The server evaluate the authentication credentials
        * If authorized: The server will return a token to the client
        * If not authorized: The server will return an error
    3. The client can use the token to access protected resources

* Each time the client tries to access the server resources:
    * The server checks the request headers
    * If the client token is found and valid:
        * client will access that resource

* There are two methods for implementing token authentication:
    1. **Token Based Authentication:** Storing the token in the database
    2. **JSON Web Token Authentication:** Storing the token in the cookie

### **10.10.** Djoser and JWT

#### **10.10.1.** Overview

* Djoser is the restful implementation for django authentication system
* Djoser provides views for registration, login, logout and more
* Djoser relies on authentication backends
    1. Django Token Based Authentication
    2. JSON Web Token Authentication

* By using the djoser API end points, we can:
    1. register new users --> /auth/users/
    2. log in and generate token --> /auth/jwt/create/
    3. getting the current user --> /auth/users/me/
    3. evaluate token --> /auth/jwt/verify/
    4. refresh token --> /auth/jwt/refresh/
    5. change password --> 
    6. forget password

#### **10.10.2.** Install and Configure

* Installing djoser and simple_jwt

In [45]:
# pipenv install djoser
# pipenv install djangorestframework_simplejwt

* Configuring djoser and jwt

In [46]:
# INSTALLED_APPS = [
    # "rest_framework",
    # "djoser",
# ]

In [47]:
# from django.urls import path, include


# urlpatterns = [
    # path(r"^auth/", include("djoser.urls")),
    # path(r"^auth/", include("djoser.urls.jwt")),
# ]

In [48]:
# REST_FRAMEWORK = {
    # 'DEFAULT_AUTHENTICATION_CLASSES': (
        # 'rest_framework_simplejwt.authentication.JWTAuthentication',
    # ),
# }

In [49]:
# SIMPLE_JWT = {
    # 'AUTH_HEADER_TYPES': ('JWT', "jwt", "Bearer", "bearer",),
# }

#### **10.10.3.** Djoser Serializer

* Djoser uses its default serializers to translate data back and forth
* These default serializer can be overwritten

In [50]:
# from djoser.serializers import UserCreateSerializer as BaseUserCreateSerializer


# class UserCreateSerializer(BaseUserCreateSerializer):
    # class Meta(BaseUserCreateSerializer.Meta):
        # fields = ["custom_field_name", "method_field_name", "model_field_name", ...]

In [51]:
# DJOSER = {
    # "SERIALIZERS": {
        # "user_create": "app_path.app_name.serializers_path.serializers.SerializerName",
    # }
# }

#### **10.10.4.** Regrading JWT

* JWT produces two tokens:
    1. access: a short lived token used for authorizing the client
    2. refresh: a long lived token used to renewing the access token when it becomes invalid

* To modify the duration of the tokens:

In [52]:
# from datetime import timedelta


# SIMPLE_JWT = {
    # "ACCESS_TOKEN_LIFETIME": timedelta(minutes=5),
    # "REFRESH_TOKEN_LIFETIME": timedelta(days=1),
# }

* Getting current user from token

In [53]:
# from django.shortcuts import get_object_or_404
# from rest_framework.response import Response
# from rest_framework.decorators import api_view
# from .models import UserModelName
# from .api.serializers import UserSerializer


# @api_view()
# def method_name_view(request):
    # user = get_object_or_404(UserModelName, pk=request.user.pk)
    # serializer = UserSerializer(user)
    # return Response(serializer.data)